In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/newsqa_train.json ./

In [ ]:
!pip install -U deep_translator

In [ ]:
import json
from deep_translator import GoogleTranslator
from tqdm import tqdm

In [ ]:
import json
data_json = json.load(open('/content/newsqa_train.json', 'r'))
len(data_json['data'])//8

In [ ]:
ws_data = {
    "version": '1.1',
    'data': []
}

In [ ]:
k=0
for data in tqdm(data_json['data'][11568:23136]):
  obj_1 = {
      "title": "",
      "paragraphs": []
  }
  for paragraph in data['paragraphs']:
    context=paragraph['context'] 
    obj_2 = {
        'context' : "",
        'qas' : []
    }
    for qa in paragraph['qas']:
      obj_3 = {
          'question': GoogleTranslator(source='auto', target='vi').translate(qa['question']),
          'answers': [],
          'id': qa['id']
      }
      for answer in qa['answers']:
        new_context = context[0:answer['answer_start']] + "[an_s] " + context[answer['answer_start']:answer['answer_start']+len(answer['text'])] + " [an_e]" + context[answer['answer_start']+len(answer['text']):]
        try:
          new_context = GoogleTranslator(source='auto', target='vi').translate(new_context)
        except:
          k=k+1
          continue
        try:
          answer_start= new_context.index("[an_s]")
          context = new_context.replace('[an_s] ', "").replace(' [an_e]','')
          text = context[answer_start:new_context.index("[an_e]")-8]
        except:
          k =k +1
          continue
        obj_4 = {
            'answer_start':answer_start,
            'text': text
        }
        obj_3['answers'].append(obj_4)
      if len(obj_3['answers']) == 0:
        continue
      obj_2['qas'].append(obj_3)
      obj_2['context'] = context
    if len(obj_2['qas']) == 0:
      continue
    obj_1['paragraphs'].append(obj_2)
  if len(obj_1['paragraphs'])==0:
    continue
  ws_data['data'].append(obj_1)

print(k)

In [ ]:
with open('/content/train_newqa_1.json', 'w') as outfile:
    json.dump(ws_data, outfile)

In [ ]:
!cp /content/train_newqa_1.json /content/drive/MyDrive